In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset
from torchvision.io import read_image
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader
import io

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cudnn.benchmark = True
plt.ion()   # interactive mode

In [23]:
def transform_image(image_bytes):
    my_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transform(image).unsqueeze(0)

In [9]:
with open("test.jpg", 'rb') as f:
    image_bytes = f.read()
    tensor = transform_image(image_bytes=image_bytes)

In [24]:
# Model class must be defined somewhere
model = torch.load("modelv1.pth")
model.to('cpu')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [36]:
def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = int(y_hat.item())
    return class_names[predicted_idx]

In [48]:
with open("download.jpg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))

japanese chin


In [29]:
#Make dictionary for our classes
test_annotation = open("annotations/annotations/test.txt").readlines()
class_id = []
label = []
for x in test_annotation:
    class_name = x.split(' ')[0].replace("_"," ")
    cleaned_name = ''.join([i for i in class_name if not i.isdigit()])
    class_id.append(cleaned_name.rstrip())
    label.append(int(x.split(' ')[1])-1)

class_names = pd.DataFrame(class_id,label).drop_duplicates().to_dict()[0]
class_names

{0: 'Abyssinian',
 1: 'american bulldog',
 2: 'american pit bull terrier',
 3: 'basset hound',
 4: 'beagle',
 5: 'Bengal',
 6: 'Birman',
 7: 'Bombay',
 8: 'boxer',
 9: 'British Shorthair',
 10: 'chihuahua',
 11: 'Egyptian Mau',
 12: 'english cocker spaniel',
 13: 'english setter',
 14: 'german shorthaired',
 15: 'great pyrenees',
 16: 'havanese',
 17: 'japanese chin',
 18: 'keeshond',
 19: 'leonberger',
 20: 'Maine Coon',
 21: 'miniature pinscher',
 22: 'newfoundland',
 23: 'Persian',
 24: 'pomeranian',
 25: 'pug',
 26: 'Ragdoll',
 27: 'Russian Blue',
 28: 'saint bernard',
 29: 'samoyed',
 30: 'scottish terrier',
 31: 'shiba inu',
 32: 'Siamese',
 33: 'Sphynx',
 34: 'staffordshire bull terrier',
 35: 'wheaten terrier',
 36: 'yorkshire terrier'}